In [ ]:
import * as tslab from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf-8");
tslab.display.html(`<style>${css}</style>`);

# The Knight's Tour

This notebook computes a solution to the [knight's tour](https://en.wikipedia.org/wiki/Knight%27s_tour) using the constraint solver `Z3`.  

In [ ]:
import { init, Bool, BitVec } from 'z3-solver';
const { Context } = await init();
const ctx = Context("main");

Given an integer from the set $\{0, 1, \cdots, 63\}$, the function `row(i)` computes the name of the variable that specifies the *row* of the knight after its $i^{\textrm{th}}$ move.

In [ ]:
function row(i: number): string {
    return `R${i}`;
}

Given an integer from the set $\{0, 1, \cdots, 63\}$, the function `col(i)` computes the name of the variable that specifies the *column* of the knight after its $i^{\textrm{th}}$ move.

In [ ]:
function col(i: number): string {
    return `C${i}`;
}

The function `isKnightMove(row, col, rowX, colX)` takes four arguments:
* `row` is a `Z3` variable that specifies the row of the position of the knight before the move.
* `col` is a `Z3` variable that specifies the column of the position of the knight before the move.
* `rowX` is a `Z3` variable that specifies the row of the position of the knight after the move.
* `colX` is a `Z3` variable that specifies the column of the position of the knight after the move.

It returns a formula that specifies that the specified position represents a legal move for a knight.

In order to form the *conjunction* of two formulas we use the function `z3.And`, 
while the *disjunction* is build with the function `z3.Or`.  Note that these functions can be called with 
any number of arguments.

The figure below shows the moves of a knight:  The knight on `e4` can jump to all red squares.
<img src="knight-moves.png" width="50%">

In [ ]:
const S = [1, 2, -1, -2];
S.flatMap(x => 
    S.filter(y => Math.abs(x) !== Math.abs(y))
     .map(y => [x, y])
);

In [ ]:
function isKnightMove(row: BitVec, col: BitVec, rowX: BitVec, colX: BitVec): Bool {
    const formulas: Bool[] = [];
    const S = [1, 2, -1, -2];
    const deltaSet: [number, number][] = [];
    for (const x of S) {
        for (const y of S) {
            if (Math.abs(x) !== Math.abs(y)) {
                deltaSet.push([x, y]);
            }
        }
    }
    for (const [delta_r, delta_c] of deltaSet) {
        const abs_dr = ctx.BitVec.val(Math.abs(delta_r), 4);
        const abs_dc = ctx.BitVec.val(Math.abs(delta_c), 4);
        let r_eq, c_eq;
        if (delta_r > 0) {
             r_eq = rowX.eq(row.add(abs_dr));
        } else {
             r_eq = rowX.add(abs_dr).eq(row);
        }
        if (delta_c > 0) {
             c_eq = colX.eq(col.add(abs_dc));
        } else {
             c_eq = colX.add(abs_dc).eq(col);
        }
        formulas.push(ctx.And(r_eq, c_eq));
    }
    return ctx.Or(...formulas);
}

The function `allDifferent` takes two arguments:
* `Rows` is a list of `Z3` variables. The variable `Rows[i]` specifies the row of the position of the knight after the $i^{\textrm{th}}$ move.
* `Cols` is a list of `Z3` variables. The variable `Cols[i]` specifies the column of the position of the knight after the $i^{\textrm{th}}$ move.

The function returns a set of formulas stating that for $i \not= j$ the positions after the $i^{\textrm{th}}$ move
differs from the position after the $j^{\textrm{th}}$ move.

In [ ]:
function allDifferent(Rows: BitVec[], Cols: BitVec[]): Bool[] {
    const result: Bool[] = [];
    for (let i = 0; i <= 62; i++) {
        for (let j = i + 1; j <= 63; j++) {
            result.push(
                ctx.Or(Rows[i].neq(Rows[j]), Cols[i].neq(Cols[j]))
            );
        }
    }
    return result;
}

The function `allConstraints` takes two arguments:
* `Rows` is a list of `Z3` variables. The variable `Rows[i]` specifies the row of the position of the knight after the $i^{\textrm{th}}$ move.
* `Cols` is a list of `Z3` variables. The variable `Cols[i]` specifies the column of the position of the knight after the $i^{\textrm{th}}$ move.

`allConstraints` returns a set containing all constraints of the problem.

In [ ]:
function allConstraints(Rows: BitVec[], Cols: BitVec[]): Bool[] {
    const constraints: Bool[] = allDifferent(Rows, Cols);
    const zero = ctx.BitVec.val(0, 4);
    const eight = ctx.BitVec.val(8, 4);
    constraints.push(Rows[0].eq(zero));
    constraints.push(Cols[0].eq(zero));
    for (let i = 0; i <= 62; i++) {
        constraints.push(
            isKnightMove(Rows[i], Cols[i], Rows[i+1], Cols[i+1])
        );
    }
    for (let i = 0; i <= 63; i++) {
        constraints.push(Rows[i].ult(eight));
        constraints.push(Cols[i].ult(eight));
    }
    return constraints;
}

The function `solve()` computes a solution of the knight's problem and returns this solution.

In [ ]:
async function solve(): Promise<Record<string, number> | null> {
    const Rows = [];
    const Cols = [];
    for (let i = 0; i <= 63; i++) {
        Rows.push(ctx.BitVec.const(row(i), 4));
        Cols.push(ctx.BitVec.const(col(i), 4));
    }
    const constraints = allConstraints(Rows, Cols);
    const solver = new ctx.Solver();
    solver.add(...constraints);
    const check = await solver.check();
    if (check === 'sat') {
        const model = solver.model();
        const solution: Record<string, number> = {};
        const parseVal = (expr: BitVec) => {
            let s = expr.toString(); 
            if (s.startsWith('#x')) {
                return parseInt(s.substring(2), 16);
            }
            return parseInt(s, 10);
        };
        
        for (let i = 0; i <= 63; i++) {
            solution[row(i)] = parseVal(model.eval(Rows[i]));
            solution[col(i)] = parseVal(model.eval(Cols[i]));
        }
        return solution;
    } else if (check === 'unsat') {
        console.log('The problem is not solvable.');
        return null;
    } else {
        console.log('Z3 cannot determine whether the problem is solvable.');
        return null;
    }
}

Unfortunately, the execution time of the following cell varies greatly between
different runs.  Sometimes the cell runs in less one minute and 28 seconds, sometimes 
it might take 30 minutes.

In [ ]:
console.time("Solver Duration");
const Solution = await solve();
console.timeEnd("Solver Duration");
Solution

The function `createBoard(Solution)` returns a matrix `Board` of size $8\times 8$.
The following holds:
$$ \texttt{Board}[\texttt{R}i][\texttt{C}i] = i $$
Therefore, if `Board[r][c] == i`, then at the beginning of the $i^{\textrm{th}}$ move the knight is located in row `r` and column `c`. 

In [ ]:
function createBoard(solution: Record<string, number>) {
    const board: number[][] = Array.from({ length: 8 }, () => Array(8).fill(0));
    for (let i = 1; i <= 63; i++) {
        const rKey = row(i);
        const cKey = col(i);
        const r = solution[rKey];
        const c = solution[cKey];
        if (typeof r === 'number' && typeof c === 'number') {
            board[r][c] = i;
        }
    }
    return board;
}

In [ ]:
if(Solution){
    createBoard(Solution);
}

The function `printBoard` prints the given `Board`.

In [ ]:
function printBoard(board: number[][]) {
    const n = board.length;
    if (n === 0) return;
    const width = board
        .flat()
        .reduce((max, element) => Math.max(max, element.toString().length), 0);
    const createLine = (left: string, mid: string, right: string, fill: string) => {
        let line = left;
        for (let i = 0; i < n - 1; i++) {
            line += fill.repeat(width + 2) + mid;
        }
        line += fill.repeat(width + 2) + right;
        return line;
    };
    const topLine = createLine('╔', '╦', '╗', '═');
    const midLine = createLine('╠', '╬', '╣', '═');
    const botLine = createLine('╚', '╩', '╝', '═');
    console.log(topLine);
    board.forEach((row, i) => {
        let line = '\u2551';
        for (const element of row) {
            const s = element.toString().padStart(width, ' ');
            line += ` ${s} ║`;
        }
        console.log(line);
        if (i < n - 1) {
            console.log(midLine);
        }
    });
    console.log(botLine);
}

In [ ]:
if(Solution){
    printBoard(createBoard(Solution));
}

# Visualization

The function `showSolution` displays the given solution on a chessboard.
The solution `Board` is represented as a list of lists.  We have `Board[row][col] == k` if the $k^\textrm{th}$ move leads the knight to the position `(row, col)`.

In [ ]:
import * as tslab from "tslab";

function showSolution(board: number[][], width: string = "50%"): void {
    const n = board.length;
    const cellSize = 50;
    const totalSize = n * cellSize;
    const path: [number, number][] = new Array(n * n);
    for (let r = 0; r < n; r++) {
        for (let c = 0; c < n; c++) {
            const k = board[r][c];
            if (k >= 0 && k < n * n) {
                path[k] = [r, c];
            }
        }
    }
    let svg = `<svg width="${width}" viewBox="0 0 ${totalSize} ${totalSize}" xmlns="http://www.w3.org/2000/svg" style="font-family: sans-serif;">`;
    for (let r = 0; r < n; r++) {
        for (let c = 0; c < n; c++) {
            const isBlack = (r + c) % 2 !== 0;
            const color = isBlack ? '#b58863' : '#f0d9b5';
            const x = c * cellSize;
            const y = r * cellSize;         
            svg += `<rect x="${x}" y="${y}" width="${cellSize}" height="${cellSize}" fill="${color}" />`;
            svg += `<text x="${x + cellSize/2}" y="${y + cellSize/2 + 5}" text-anchor="middle" font-size="14" fill="${isBlack ? '#f0d9b5' : '#b58863'}">${board[r][c]}</text>`;
        }
    }
    if (path[0]) {
        const [startR, startC] = path[0];
        svg += `<circle cx="${startC * cellSize + cellSize/2}" cy="${startR * cellSize + cellSize/2}" r="8" fill="darkblue" />`;
    }
    for (let k = 0; k < n * n - 1; k++) {
        const p1 = path[k];
        const p2 = path[k+1];
        if (p1 && p2) {
            const x1 = p1[1] * cellSize + cellSize/2;
            const y1 = p1[0] * cellSize + cellSize/2;
            const x2 = p2[1] * cellSize + cellSize/2;
            const y2 = p2[0] * cellSize + cellSize/2;
            svg += `<line x1="${x1}" y1="${y1}" x2="${x2}" y2="${y2}" stroke="darkblue" stroke-width="3" stroke-opacity="0.6" />`;
            svg += `<circle cx="${x2}" cy="${y2}" r="3" fill="darkblue" />`;
        }
    }
    svg += `</svg>`;
    tslab.display.html(svg);
}

In [ ]:
if(Solution){
    showSolution(createBoard(Solution));
}